### Question 1

In [2]:
import pyspark
from pyspark.sql import SparkSession, types

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('homework') \
    .getOrCreate()
    
spark.version

your 131072x1 screen size is bogus. expect trouble
25/03/17 21:40:05 WARN Utils: Your hostname, DESKTOP-Q5PPUKU resolves to a loopback address: 127.0.1.1; using 172.30.227.126 instead (on interface eth0)
25/03/17 21:40:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 21:40:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/17 21:40:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


'3.5.5'

### Question 2

In [33]:
import os
import pyspark
import requests
import numpy as np
import pyarrow.parquet as pq
from pyspark.sql import SparkSession, types

init_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet"

filename = "yellow_tripdata_2024-10.parquet"

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('homework') \
    .getOrCreate()

with requests.get(init_url) as response:
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        df = spark.read.parquet(filename)
        df.repartition(4).write.parquet(f'./parquet')
files = [f for f in os.listdir("./parquet/") if f.endswith(".parquet")]
size = []
for file in files:
    size.append(os.path.getsize(f'./parquet/{file}'))
size = np.array(size)
print(f'{round(size.mean()/(1024 * 1024), 2)}MB')
### remove file parquet from url
os.remove(filename)
        

22.4MB


### Question 3

In [54]:
import os
import pyspark
import requests
import numpy as np
import pyarrow.parquet as pq
from pyspark.sql import SparkSession, types

init_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet"

filename = "yellow_tripdata_2024-10.parquet"

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('homework') \
    .getOrCreate()
    
with requests.get(init_url) as response:
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        df = spark.read.parquet(filename)
        df.createOrReplaceTempView('yellow_trip')
        spark.sql("""
                  SELECT 
                    extract(day from tpep_pickup_datetime) as day,
                    count(*) AS total_trips
                  FROM yellow_trip WHERE tpep_pickup_datetime >= '2024-10-15 00:00:00' AND tpep_pickup_datetime < '2024-10-16 00:00:00' GROUP BY 1 ORDER BY day;
                  """).show()

os.remove(filename)

+---+-----------+
|day|total_trips|
+---+-----------+
| 15|     128893|
+---+-----------+



### Question 4

In [ ]:
import os
import pyspark
import requests
import numpy as np
import pyarrow.parquet as pq
from pyspark.sql import SparkSession, types

init_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet"

filename = "yellow_tripdata_2024-10.parquet"

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('homework') \
    .getOrCreate()
    
with requests.get(init_url) as response:
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        df = spark.read.parquet(filename)
        df.createOrReplaceTempView('yellow_trip')
        spark.sql("""
                  SELECT 
                    TIMESTAMPDIFF(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime) AS hour_difference
                  FROM yellow_trip ORDER BY hour_difference DESC;
                  """).show()

os.remove(filename)

+---------------+
|hour_difference|
+---------------+
|            162|
|            143|
|            137|
|            114|
|             89|
|             89|
|             70|
|             67|
|             66|
|             46|
|             42|
|             38|
|             33|
|             26|
|             25|
|             25|
|             24|
|             23|
|             23|
|             23|
+---------------+
only showing top 20 rows



### Question 6

In [65]:
import os
import pyspark
import requests
import numpy as np
import pyarrow.parquet as pq
from pyspark.sql import SparkSession, types

yellow_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet"
taxi_zone_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

yellow_file = "yellow_tripdata_2024-10.parquet"
taxi_zone_file = "taxi_zone_lookup.csv"

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('homework') \
    .getOrCreate()
    
with requests.get(yellow_url) as response:
    if response.status_code == 200:
        with open(yellow_file, 'wb') as file:
            file.write(response.content)

with requests.get(taxi_zone_url) as response:
    if response.status_code == 200:
        with open(taxi_zone_file, 'wb') as file:
            file.write(response.content)
            
yellow_df = spark.read.parquet(yellow_file)
taxi_zone = spark.read \
            .option("header", "true") \
            .csv(taxi_zone_file)
yellow_df.createOrReplaceTempView('yellow_trip')
taxi_zone.createOrReplaceTempView('taxi_zone')
spark.sql("""
            WITH fct_yellow_trip AS (SELECT VendorID,
                                            tpep_pickup_datetime as pickup_datetime,
                                            tpep_dropoff_datetime as dropoff_datetime,
                                            passenger_count,
                                            trip_distance,
                                            RatecodeID,
                                            store_and_fwd_flag,
                                            PULocationID,
                                            DOLocationID,
                                            payment_type,
                                            fare_amount,
                                            extra,
                                            mta_tax,
                                            tip_amount,
                                            tolls_amount,
                                            improvement_surcharge,
                                            total_amount,
                                            congestion_surcharge,
                                            Airport_fee
                                    FROM yellow_trip),
            dim_taxi_zone AS (SELECT locationid, borough, zone, replace(service_zone, 'Boro', 'Green') as service_zone FROM taxi_zone),
            final_table AS (
            SELECT 
                y.VendorID,
                y.pickup_datetime,
                y.dropoff_datetime,
                y.passenger_count,
                y.trip_distance,
                y.RatecodeID,
                y.store_and_fwd_flag,
                y.PULocationID,
                pz.borough as pickup_borough,
                pz.zone as pickup_zone,
                y.DOLocationID,
                dz.borough as dropoff_borough,
                dz.zone as dropoff_zone,
                y.payment_type,
                y.fare_amount,
                y.extra,
                y.mta_tax,
                y.tip_amount,
                y.tolls_amount,
                y.improvement_surcharge,
                y.total_amount,
                y.congestion_surcharge,
                y.Airport_fee
            FROM fct_yellow_trip AS y
            INNER JOIN dim_taxi_zone AS pz
            ON (y.PULocationID = pz.locationid)
            INNER JOIN dim_taxi_zone AS dz
            ON (y.DOLocationID = dz.locationid)
            )
            SELECT pickup_zone,
            count(*) AS total_pickup
            FROM final_table GROUP BY 1 ORDER BY 2
            """).show()

os.remove(filename)

+--------------------+------------+
|         pickup_zone|total_pickup|
+--------------------+------------+
|Governor's Island...|           1|
|       Rikers Island|           2|
|       Arden Heights|           2|
|         Jamaica Bay|           3|
| Green-Wood Cemetery|           3|
|Charleston/Totten...|           4|
|   Rossville/Woodrow|           4|
|       West Brighton|           4|
|Eltingville/Annad...|           4|
|       Port Richmond|           4|
|         Great Kills|           6|
|        Crotona Park|           6|
|Heartland Village...|           7|
|     Mariners Harbor|           7|
|Saint George/New ...|           9|
|             Oakwood|           9|
|       Broad Channel|          10|
|New Dorp/Midland ...|          10|
|         Westerleigh|          12|
|     Pelham Bay Park|          12|
+--------------------+------------+
only showing top 20 rows

